In [89]:
import pandas as pd
import numpy as np

In [2]:
customers = pd.read_csv('data/olist_customers_dataset.csv', sep=',')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv', sep=',')
order_items = pd.read_csv('data/olist_order_items_dataset.csv', sep=',')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv', sep=',')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv', sep=',')
orders = pd.read_csv('data/olist_orders_dataset.csv', sep=',')
products = pd.read_csv('data/olist_products_dataset.csv', sep=',')
sellers = pd.read_csv('data/olist_sellers_dataset.csv', sep=',')

In [94]:
order_payments_merge = orders.merge(order_payments, how="left", on="order_id", validate="1:m").groupby('order_id').agg(
    {"customer_id": "first",
    "order_purchase_timestamp": "first",
    "payment_value": "sum"})

In [96]:
order_payments_merge.head()

,customer_id,order_purchase_timestamp,payment_value
order_id,,,
00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,72.19
00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,259.83
000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,216.87
00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,218.04


In [72]:
order_payments_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99441 entries, 00010242fe8c5a6d1ba2dd792cb16214 to fffe41c64501cc87c801fd61db3f6244
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               99441 non-null  object 
 1   order_purchase_timestamp  99441 non-null  object 
 2   payment_value             99441 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.0+ MB


In [73]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [97]:
rfm = customers.merge(order_payments_merge, how="left", on="customer_id", validate="1:1").groupby('customer_unique_id').agg(
    recence = ("order_purchase_timestamp", "max"),
    frequence = ("customer_id", "count"),
    montant_cumulé = ("payment_value", "sum")
)

In [98]:
rfm.head()

,recence,frequence,montant_cumulé
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19
0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62
0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89


In [100]:
rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 96096 entries, 0000366f3b9a7992bf8c76cfdf3221e2 to ffffd2657e2aad2907e67c3e9daecbeb
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   recence         96096 non-null  object 
 1   frequence       96096 non-null  int64  
 2   montant_cumulé  96096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.9+ MB


In [102]:
rfm[rfm['frequence'] > 1].count()*100 / rfm.shape[0]

recence           3.118756
frequence         3.118756
montant_cumulé    3.118756
dtype: float64